In [ ]:
from Datos import Datos
from EstrategiaParticionado import ValidacionSimple
from EstrategiaParticionado import ValidacionCruzada
from Clasificador import ClasificadorNaiveBayes,Clasificador
from Verificador import Verificador_GaussianNB, Verificador_Multinominal
from MatrizConfusion import MatrizConfusion

from tabulate import tabulate
import sys

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB


In [ ]:
"""
Apartado 1 => Particionado

Análisis de las dos estrategias de particionado propuestas: simple, y
cruzada, para los conjuntos propuestos: german y tic-tac-toe. El análisis
consiste en una descripción de los índices de train y test devueltos por
cada uno de los métodos de particionado, junto con un comentario
sobre las ventajas/desventajas de cada uno de ellos.
"""

In [ ]:
"""
En cuanto al metodo de VALIDACIÓN SIMPLE es el mas sencillo de implementar, puesto que se basa en una division 
aleatoria en dos grupos. Sin embargo, aún siendo el modo de validación más sencillo, trae consigo una serie de
inconvenientes:
    
    1) El ratio de error, es altamente variable dependiendo de las instancias del dataset escogido, para 
    entrenamiento, y test.
    
    2) Al segregar una serie de instancias para entrenamiento y test, estamos provocando que durante el entrenamiento
    el clasificador, no contemple todas las posibles situaciones de un contexto, por lo tanto esto produce
    una sobrestimación del ratio de error.
    
Por su parte el método de VALIDACIÓN CRUZADA es más complejo de implementar, puesto que se trata de un proceso 
iterativo. Éste, busca dividir todo el conjunto de datos en k grupos, de tal modo que, solo uno de esos bloques se usa 
como test, mientras que el resto de divisiones se usan como entrenamiento. Por cada iteración se turnan los "roles", de 
tal modo, que todas las subdivisiones son testadas y todas han sido usadas como entrenamiento. 

En un escenario, en el que el conjunto de datos es pequeño, es muy superior a validacón simple y otra ventaja 
respecto a validación simple es que, prueba y valida todas las instancias de datos. Por otra parte, podemos decir que
el ratio de error en este tipo de validacion es mucho más preciso y real, puesto que se entrena con prácticamente todo
el conjunto de datos maximizando asi el modelo sin sobrestimar en las predicciones. A su vez, es de gran
utilidad el que el ratio de error sea calculado como un promedio de las estimaciones de cada iteración.

Respecto los inconvenientes, principalmente encontramos uno, que es el coste computacional que requiere este método.
La identificación de bloques, el número de iteraciones y sobre todo que en el caso de usar un dataset 
excesivamente grande, debemos contemplar la posibilidad de que se vuelva algo lento y costoso (en comparación con
validación simple) y por tanto no sea tan útil como en otras circunstancias.
"""

In [ ]:
fileName = "ConjuntosDatos/tic-tac-toe.data"
datos_tic = Datos(fileName)
    
    # Probamos con 75 porciento y 10 iteraciones (Validacion Simple)
validacion_simple_tic = ValidacionSimple(75,10)
aux_simple_tic = validacion_simple_tic.creaParticiones(datos_tic)

print("######## VALIDACION SIMPLE TIC-TAC-TOE ########")
print("Porcentaje entrenamiento: " + str(validacion_simple_tic.porcentaje))
print("Numero iteraciones: " + str(validacion_simple_tic.numEjecuciones) + " <=> Numero particiones: " + str(len(aux_simple_tic)))
print("Indices Totales(Primera Iteracion/Particion) => " + str(datos_tic.cantidadDatos))
print("\tNum Indices Train: " + str(len(aux_simple_tic[0].indicesTrain)))
print("\tNum Indices Test: " + str(len(aux_simple_tic[0].indicesTest)))

    # Probamos con 10 k-iteraciones
validacion_cruzada_tic = ValidacionCruzada(6)
aux_cruzada_tic = validacion_cruzada_tic.creaParticiones(datos_tic)

print("\n######## VALIDACION CRUZADA TIC-TAC-TOE ########")
print("Numero Carpetas: " + str(validacion_cruzada_tic.numParticiones) + " <=> Numero particiones: " + str(len(aux_cruzada_tic)))
print("Indices Totales(Primera Iteracion/Particion) => " + str(datos_tic.cantidadDatos))
print("\tNum Indices Train: " + str(len(aux_cruzada_tic[0].indicesTrain)))
print("\tNum Indices Test: " + str(len(aux_cruzada_tic[0].indicesTest)))

fileName = "ConjuntosDatos/german.data"
datos_ger = Datos(fileName)

    # Probamos con 75 porciento y 10 iteraciones (Validacion Simple)
validacion_simple_ger = ValidacionSimple(75,10)
aux_simple_ger = validacion_simple_ger.creaParticiones(datos_ger)

print("\n######## VALIDACIONES GERMAN ########")
print("Porcentaje entrenamiento: " + str(validacion_simple_ger.porcentaje))
print("Numero iteraciones: " + str(validacion_simple_ger.numEjecuciones) + " <=> Numero particiones: " + str(len(aux_simple_ger)))
print("Indices Totales(Primera Iteracion/Particion) => " + str(datos_ger.cantidadDatos))
print("\tNum Indices Train: " + str(len(aux_simple_ger[0].indicesTrain)))
print("\tNum Indices Test: " + str(len(aux_simple_ger[0].indicesTest)))

# Probamos con 10 k-iteraciones
validacion_cruzada_ger = ValidacionCruzada(6)
aux_cruzada_ger = validacion_cruzada_ger.creaParticiones(datos_ger)
print("\n######## VALIDACION CRUZADA GERMAN ########")
print("Numero Carpetas: " + str(validacion_cruzada_ger.numParticiones) + " <=> Numero particiones: " + str(len(aux_cruzada_ger)))
print("Indices Totales(Primera Iteracion/Particion) => " + str(datos_ger.cantidadDatos))
print("\tNum Indices Train: " + str(len(aux_cruzada_ger[0].indicesTrain)))
print("\tNum Indices Test: " + str(len(aux_cruzada_ger[0].indicesTest)))

In [ ]:
"""
Como vemos, hemos representado dos particiones para cada dataset empleado en la práctica.
Las validaciones simples hemos almacenado el numero de iteraciones que para ambas pruebas ha sido de 10.
En cada iteración, que corresponde a su vez a una particion, almacenamos de modo aleatorio el número de indices 
especificado por el usuario en forma de porcentaje (para nuestro ejemplo, se trata de un 0.75).

En cuanto a la validación cruzada, preparamos a priori, cada bloque con el número de indices que le corresponde.
Para el caso de tic-tac-toe.data, estamos usando un total de 958 entradas en el datset que se reparten entre 6 bloques
con 158-160 indices. Como vemos el número de indices varia, dependiendo de la situación y de si el total de indices, es
divisible (con resto 0) wntre el número de bloques (Kfolds) que escogio el usuario.

Tras ello, procedemos a insertar los indices de cada bloque en los array's de indicesTrain, e indicesTest, y tenemos en
cuenta, que bloque ha sido de test en cada iteración, para en cada una, escoger uno diferente y asignar el resto de
bloques como datos de train.
"""

In [ ]:
"""
Apartado 2 => Naive Bayes

Tabla con los resultados de la ejecución para los conjuntos de datos
analizados (tic-tac-toe y german). Considerar los dos tipos de
particionado.
Los resultados se refieren a las tasas de error/acierto y deben incluirse
tanto con la corrección de Laplace como sin ella. Se debe incluir tanto
el promedio de error para las diferentes particiones como su desviación
típica. Es importante mostrar todos los resultados agrupados en una
tabla para facilitar su evaluación.
Breve análisis de los resultados anteriores.
"""

In [ ]:
"""
El objetivo de este apartado es crear el modelos Naive Bayes que se entrene con los ejemplos indexados  por 'train' generados en el apartado anterior y que clasifique aquellos marcados por los indices 'test' tambien porporcionados por los validadores.

El modo de clasificación de Naive Bayes trae consigo una premisa que hace que su implementación sea más sencilla. Esta es la suposición de que los atributos del dataset son independientes unos de otros dada la clase. 

Como ventaja, la independencia de atributos, nos trae que se puede crear un modelo potente, con relativamente poco coste computacional y que es facilmente escalable en cuanto a atributos se refiere. Esto quiere decir que el coste computacional es lineal.

Como inconveniente, la premisa de la que se parte es demasiado optimista, pues en muchas ocasiones ciertos atributos del dataset pueden estar relacionados con otros. Por ejemplo en un dataset con un atributo "nubes" (sí/no) y otro "lluvia" (sí/no) hay una clara relación puesto que si no hay nubes tampoco hay lluvia. Con esto se quiere decir que un modelo que tenga en cuentas esta dependecia de atributos será más preciso que NB, o lo que es lo mismo, NB pierde precisión al suponer la independecia entre atributos.

"""

In [ ]:
# Creacion del clasificador
Clasificador = ClasificadorNaiveBayes()

#       Sin Laplace
#  TIC-TAC-TOE
# Validacion con validacion simple
media_error1, media_tp1, media_fp1, media_tn1, media_fn1 = Clasificador.validacion(validacion_simple_tic,datos_tic,False)

# Validacion con validacion cruzada
media_error2, media_tp2, media_fp2, media_tn2, media_fn2 = Clasificador.validacion(validacion_cruzada_tic,datos_tic,False)

#  GERMAN  
# Validacion con validacion simple
media_error3, media_tp3, media_fp3, media_tn3, media_fn3 = Clasificador.validacion(validacion_simple_ger,datos_ger,False)

# Validacion con validacion cruzada
media_error4, media_tp4, media_fp4, media_tn4, media_fn4 = Clasificador.validacion(validacion_cruzada_ger,datos_ger,False)

# Impresion de los resultados
resultados_sin = [[round(media_error1, 3), round(media_error3, 3)], [round(media_error2, 3), 
round(media_error4, 3)]]

print("Sin Laplace")
print(tabulate(resultados_sin, headers=['Tasa de error', 'Tic-Tac-Toe', 'German'], showindex=['Val. Simple', 'Val. Cruzada'], tablefmt='fancy_grid'))

#       Con Laplace
#  TIC-TAC-TOE
# Validacion con validacion simple
media_error1, media_tp1, media_fp1, media_tn1, media_fn1 = Clasificador.validacion(validacion_simple_tic,datos_tic,True)

# Validacion con validacion cruzada
media_error2, media_tp2, media_fp2, media_tn2, media_fn2 = Clasificador.validacion(validacion_cruzada_tic,datos_tic,True)

#  GERMAN  
# Validacion con validacion simple
media_error3, media_tp3, media_fp3, media_tn3, media_fn3 = Clasificador.validacion(validacion_simple_ger,datos_ger,True)

# Validacion con validacion cruzada
media_error4, media_tp4, media_fp4, media_tn4, media_fn4 = Clasificador.validacion(validacion_cruzada_ger,datos_ger,True)

# Impresion de los resultados
resultados_con = [[round(media_error1, 3), round(media_error3, 3)], [round(media_error2, 3), 
round(media_error4, 3)]]
print("Con Laplace")
print(tabulate(resultados_con, headers=['Tasa de error', 'Tic-Tac-Toe', 'German'], showindex=['Val. Simple', 'Val. Cruzada'], tablefmt='fancy_grid'))


In [ ]:
"""
 Como se puede observar en las tablas generadas, se cumple lo dicho en el apartado anterior, 
 siendo el error medio de la validación cruzada más bajo que el de la simple. De lo cual se puede deducir que la validación cruzada es más precisa.

 Del mismo modo, podemos observar que en ambos tipos de validación, el error medio de 'german.data' es inferior al de tic-tac-toe'. Esto se debe a que, al tener más ejemplos con los que entrenar, el modelo es más preciso y clasifica mejor los ejemplos de train.

 Por último, en estos datasets pequeños, no se aprecia una gran diferencia entre la ejecición sin y con la correccion de Laplace. Esto se debe a que para mostrar los datos, se han redondeado a 3 decimales, sin embargo, si nos fijasemos en más decimales, veríamos que con la corrección el error es sutilmente más pequeño.
"""

In [ ]:
"""
Aparto 3 => Scikit-Learn
Incluir los mismos resultados que en el apartado 2 pero usando los
métodos del paquete scikit-learn. Comparar y analizar los resultados. 
"""

In [ ]:
""" 
 Para llevar a cabo la verificación de nuestros clasificadores implementados usamos la libreria scikitlearn, 
 de la cual usamos varias funciones de clasificación, varias de metodos de validación y varias de preprocesado de datos.
 Para empezar, hemos desarrollado dos funciones propias de la clase abstracta Verificador, 
 las cuales se encargana de llevar a cabo un preprocesado similar al que desarrollamos en la primera entrega, 
 y por otra parte hemos desarrollado un preprocesado OneHot que nos ha proporcionado muy buenos resultados. 
 
 Por otro lado, implementamos una función muy pequeña que se encarga de separar los datos de entrada y los de clasificación
 y almacenarnos en estructuras diferentes. 
 Por último, como funciones abstractas desarrollamos dos métodos de validación (validación simple y validación cruzada).

 En cuanto a las subclases que encontramos, hemos decidido implementar tanto el clasififcador GaussianNB como el MultinominalNB. 
 Ambas se pueden configurar mediante hiperparametros, y aunque ambas nos han devuelto resultados muy buenos 
 en la siguiente implementación hemos decidido usar únicamente el GaussianNB, puesto que es el que mejor ha 
 respondido a los datasets proporcinados.
"""

In [ ]:
vgNB = Verificador_GaussianNB()
# GaussianNB => Sin Preprocesado
#     TIC-TAC-TOE => Validacion Simple (TRAIN=0.75)
tic_simple_sin = vgNB.clasificate(prepro=False,tipo_validacion=1,porcentaje=0.75,folds=3,archivo="ConjuntosDatos/tic-tac-toe.data")
#     TIC-TAC-TOE => Validacion Cruzada (KFOLDS = 6)
tic_cruzada_sin = vgNB.clasificate(prepro=False,tipo_validacion=2,porcentaje=0.75,folds=6,archivo="ConjuntosDatos/tic-tac-toe.data")

#     GERMAN => Validacion Simple (TRAIN=0.75)
german_simple_sin = vgNB.clasificate(prepro=False,tipo_validacion=1,porcentaje=0.75,folds=3,archivo="ConjuntosDatos/german.data")
#     GERMAN => Validacion Cruzada (KFOLDS = 6)
german_cruzada_sin = vgNB.clasificate(prepro=False,tipo_validacion=2,porcentaje=0.75,folds=6,archivo="ConjuntosDatos/german.data")

# Impresion de los resultados
resultados_sk_sin = [[round(tic_simple_sin, 3), round(german_simple_sin, 3)], [round(tic_cruzada_sin, 3), round(german_cruzada_sin, 3)]]



# GaussianNB => Con Preprocesado

#     TIC-TAC-TOE => Validacion Simple (TRAIN=0.75)
tic_simple_con = vgNB.clasificate(prepro=True,tipo_validacion=1,porcentaje=0.75,folds=3,archivo="ConjuntosDatos/tic-tac-toe.data")
#     TIC-TAC-TOE => Validacion Cruzada (KFOLDS = 6)
tic_cruzada_con = vgNB.clasificate(prepro=True,tipo_validacion=2,porcentaje=0.75,folds=6,archivo="ConjuntosDatos/tic-tac-toe.data")

#     GERMAN => Validacion Simple (TRAIN=0.75)
german_simple_con = vgNB.clasificate(prepro=True,tipo_validacion=1,porcentaje=0.75,folds=3,archivo="ConjuntosDatos/german.data")
#     GERMAN => Validacion Cruzada (KFOLDS = 6)
german_cruzada_con = vgNB.clasificate(prepro=True,tipo_validacion=2,porcentaje=0.75,folds=6,archivo="ConjuntosDatos/german.data")

# Impresion de los resultados
resultados_sk_con = [[round(tic_simple_con, 3), round(german_simple_con, 3)], [round(tic_cruzada_con, 3), round(german_cruzada_con, 3)]]

# Impresion de las tablas
print("Practica 1:")
print(" Sin Laplace")
print(tabulate(resultados_sin, headers=['Tasa de error', 'Tic-Tac-Toe', 'German'], showindex=['Val. Simple', 'Val. Cruzada'], tablefmt='fancy_grid'))
print(" Con Laplace")
print(tabulate(resultados_con, headers=['Tasa de error', 'Tic-Tac-Toe', 'German'], showindex=['Val. Simple', 'Val. Cruzada'], tablefmt='fancy_grid'))

print("\nSKLearn:")
print(" Sin Preprocesado")
print(tabulate(resultados_sk_sin, headers=['Tasa de error', 'Tic-Tac-Toe', 'German'], showindex=['Val. Simple', 'Val. Cruzada'], tablefmt='fancy_grid'))
print(" Con Preprocesado")
print(tabulate(resultados_sk_con, headers=['Tasa de error', 'Tic-Tac-Toe','German'], showindex=['Val. Simple', 'Val. Cruzada'], tablefmt='fancy_grid'))


In [ ]:
"""
 Para validar los resultados debemos conocer la importancia y efectividad del preprocesado OneHot, ideal para tablas
 dataset's que albergan pocas columnas. Este tipo de preprocesado usa las columnas y sus posibles valores para reconstruir el
 dataset con tantas columnas como [Sumatorio{Columna(i) * #Valores(Columna(i))}]. Esto nos permite que teniendo el mismo
 numero de entradas en el dataset, podamos tener una codificacion mucho más concreta y por ende facilitemos la labor al
 clasificador puesto que ahora las entradas se codifican de modo binario. 
 
 Conociendo esta situación vemos como todas aquellas pruebas realizadas tanto en un fichero como en otro, son mucho
 mejores que sin aplicar un preprocesado OneHot. Por otra parte vemos como la validación simple es mucho mejor en cuanto
 al error medio presentado con validacion cruzada y además vemos que la clasificación en general se realiza de mejor manera
 para el fichero tic-tac-toe.dat que para el fichero german.data, posiblemente porque este ultimo contiene más registros
 y más columnas.
 
 Como hiperparametros, hemos empleado 0.75 para el entrenamiento en validación simple y 6 Kfolds para validación cruzada.
 En cuanto al primero, consideramos un porcentaje de entrenamiento lo suficientemente grande como para no sobreestimar en
 la clasififcación y para asi garantizar esa capacidad de geeneralizar frente los datos a clasificar.
 
 En cuanto al segundo hiperparametro lo consideramos algo pequeño, para estos dataset. Hemos probado a aumentar este número
 de bloques y hemos observado que a medida que aumentaba el error medio disminuia. Podemos afirmar, que para tic-tac-toe.data
 un "correcto" número de carpetas sería 12, mientras que para german.data este sería de 10 carpetas.
 
 Aún con estas modificaciones, error nos disminuye considerablemente, pero si consideramos un método de estimar cuantas
 carpetas son necesarias dado el volumen y las condiciones de un dataset.
"""

In [ ]:
"""
Apartado 4 => Evaluación de hipótesis mediante Análisis ROC
Matriz de confusión y diagramas del clasificador en el espacio ROC
"""

In [ ]:
"""
La matriz de confusión toma los valores medios de las ejecuciones del modelo con correccion de Laplace, tanto las de validación simple como las de cruzada. Con ella, podemos comprobar de una manera mucho más precisa la precisión de nuestro modelo. Se pueden ver los aciertos y los fallos que ha tenido al clasificar la clase 1 como 2 y viceversa. El formato escogido el es siguiente:

TPR FPR
FNR TNR

De esta forma, de un vistazo a la diagonal normal se pueden ver los aciertos que hemos tenido.

En cuanto al espacio ROC, es una forma más visual de comprobar el comportamiento del modelo. Hemos generado también una recta entre (0,0) y (1,1) con el fin de que, de un vistazo, se pueda ver el nivel de aciertos y de fallos. Si el punto generado está por encima de la recta, el modelo acierta más que falla, por lo tanto de podría decir que es un buen modelo. Si, por el contrario, el punto se encuentra por debajo de la recta, nuestro modelo falla más que acierta y, en consecuencia, no sería válido. 

"""

In [ ]:
# Calculamos la media de las tasas de val. simple y val. cruzada para la matriz de confusion media
    mx1 = MatrizConfusion()

    # TIC-TAC-TOE
    print("\nTic-Tac-Toe\n")
    tpr, fpr = mx1.matrix_media(media_tp1, media_tp2, media_fp1, media_fp2, 
                    media_tn1, media_tn2, media_fn1, media_fn2)
    plot_points = [[fpr, tpr, 'NB']]
    mx1.plot(plot_points, "tic-tac-toe")

    # GERMAN
    print("\nGerman\n")

    tpr, fpr = mx1.matrix_media(media_tp3, media_tp4, media_fp3, media_fp4, 
                    media_tn3, media_tn4, media_fn3, media_fn4)
    plot_points = [[fpr, tpr, 'NB']]
    mx1.plot(plot_points, "german")

In [ ]:
"""
En cuanto a la matriz de confusión, podemos observar que en ambos casos la clase positiva es mucho más abundante en el dataset que la negativa, por tanto los números de la primera fila son más altos. Sin embargo, cabe destacar que la cantidad de aciertos en ambas es bastante mayor que el de fallos,lo que se traduce en una predicción por encima de la diagonal normal.

Se observa también que el porcentaje de aciertos/fallos de la clase 1 (positiva) es mucho menor que el de la clase 2 (negativa). Esto se debe a que, como se ha mencionado anteriormente, en el dataset existen más ejemplos de la primera clase que de la segunda y como consecuencia, el modelo se encuentra mejor entrenado en la clase positiva y acierta más en esta. 

"""